In [13]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
# from keras.models import Sequential, Model
import keras.models
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

## Engineering Features

In [21]:
feat = os.path.join("bird_data","features")
mfcc_train_folder = os.path.join(feat,"mfcc_features", "train")
mfcc_test_folder = os.path.join(feat,"mfcc_features", "test")

# Neural Network

In [22]:
# Batch and target size
BATCH_SIZE = 32
TARGET_SIZE = (288,288)


In [35]:
# model
model = keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(288, 288, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 286, 286, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 284, 284, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 142, 142, 64)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 142, 142, 64)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1290496)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                82591808  
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)               

In [36]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping]

In [37]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip = False)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [38]:
test_set = test_datagen.flow_from_directory(mfcc_test_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')

Found 200 images belonging to 10 classes.


In [39]:
training_set = train_datagen.flow_from_directory(mfcc_train_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical', shuffle=True)

Found 800 images belonging to 10 classes.


val_set = test_datagen.flow_from_directory(mfcc_val_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')

In [40]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(training_set.classes), training_set.classes)
weights = {label:w for label, w in zip(np.unique(training_set.classes), class_weights)}

In [41]:
model.fit(training_set, epochs=10, class_weight=weights,  validation_data = test_set, verbose=True, callbacks=callbacks, shuffle=True)

Epoch 1/10
25/25 [==============================] - 73s 3s/step - loss: 33.4915 - accuracy: 0.0997 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/10
25/25 [==============================] - 73s 3s/step - loss: 2.3020 - accuracy: 0.1001 - val_loss: 2.2950 - val_accuracy: 0.1250
Epoch 3/10
25/25 [==============================] - 74s 3s/step - loss: 2.2963 - accuracy: 0.1219 - val_loss: 2.2970 - val_accuracy: 0.1150
Epoch 4/10
25/25 [==============================] - 73s 3s/step - loss: 2.2999 - accuracy: 0.1032 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/10
25/25 [==============================] - 74s 3s/step - loss: 2.3023 - accuracy: 0.1095 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 6/10
25/25 [==============================] - 74s 3s/step - loss: 2.3022 - accuracy: 0.1011 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 7/10
25/25 [==============================] - 75s 3s/step - loss: 2.3029 - accuracy: 0.0777 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 8/10
25/25 [=

In [42]:
model.save('Bird_Classifier.h5')

Y_pred = model.predict_generator(val_set)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(val_set.classes, y_pred))

In [44]:
score = model.evaluate(training_set, verbose=0)
print("Training Accuracy: ", score[1])

Training Accuracy:  0.10000000149011612


In [45]:
score = model.evaluate(test_set, verbose=0)
print("Testing Accuracy: ", score[1])

Testing Accuracy:  0.10000000149011612


score = model.evaluate_generator(val_set, verbose=0)
print("Validating Accuracy: ", score[1])